In [64]:
from fastai.imports import *
from fastai.structured import *
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics
import feather
import pdpbox.pdp as pdp

from plotnine import *

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import gc
import os

plt.rcParams["figure.figsize"] = (12, 10)
set_plot_sizes(12, 14, 16)
np.set_printoptions(precision=5)
pd.set_option("display.precision", 5)
%precision 5

# constants #################################################################
PATH = 'data/bulldozers/'


# defines: ##################################################################
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns",
                           1000):
        display(df)


def split_vals(a, n):
    return a[:n].copy(), a[n:].copy()


def rmse(x, y):
    return math.sqrt(((x - y)**2).mean())


def do_print_score(m, X_train, y_train, X_valid, y_valid):
    res = [
        rmse(m.predict(X_train), y_train),
        rmse(m.predict(X_valid), y_valid),
        m.score(X_train, y_train),
        m.score(X_valid, y_valid),
    ]
    if hasattr(m, "oob_score_"):
        res.append(m.oob_score_)
    print(np.array(res))

In [65]:
!ls {PATH}

'Data Dictionary.xlsx'	 Train.csv	     median_benchmark.csv
 Machine_Appendix.csv	 Valid.csv	     random_forest_benchmark_test.csv
 Test.csv		 ValidSolution.csv


In [67]:
%%time
df_raw = pd.read_csv(
    f'{PATH}Train.csv', low_memory=False, parse_dates=["saledate"])

df_raw.SalePrice = np.log(df_raw.SalePrice)
add_datepart(df_raw, 'saledate')
train_cats(df_raw)
df_raw.UsageBand = df_raw.UsageBand.cat.codes

os.makedirs('tmp', exist_ok=True)
df_raw.to_feather('tmp/bulldozers-raw')

CPU times: user 57.7 s, sys: 277 ms, total: 57.9 s
Wall time: 56.9 s


In [95]:
%%time
df, y, nas = proc_df(df_raw, 'SalePrice')

# m = RandomForestRegressor(n_jobs=-1, n_estimators=10)
# m.fit(df, y)
# print('%.5f' % m.score(df, y))

CPU times: user 2.95 s, sys: 99.8 ms, total: 3.05 s
Wall time: 541 ms
